In [1]:
# Imports
from dotenv import load_dotenv
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import LlamaCpp, OpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.vectorstores import FAISS
from tqdm import tqdm

import os
import pickle

In [2]:
# Load OpenAI's API key
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# Load documents' texts
with open("../dataset/documents.pkl", "rb") as f:
    docs = pickle.load(f)

In [4]:
# Load queries' text
with open("../dataset/queries.pkl", "rb") as f:
    queries = pickle.load(f)

In [5]:
# Load ground truth
with open("../dataset/rel_set.pkl", "rb") as f:
    rel_set = pickle.load(f)

### Common Setup

In [6]:
k = 10  # Choose the top-k most relevant documents

In [7]:
# Load FAISS index from file
faiss_vs = FAISS.load_local(
    folder_path="../ir_techniques/faiss/",
    embeddings=OpenAIEmbeddings())
retriever = faiss_vs.as_retriever(search_kwargs={"k": k})

In [8]:
def format_docs(_docs):
    ls = []
    for doc in _docs:
        if int(doc.page_content) in docs:
            ls.append(docs[int(doc.page_content)]["text"])
    return ls

In [9]:
# Define the prompt for the LLM
template = """Answer the question or Explain the topic given this additional context: {context}
Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

### da-vinci-0.0.3

In [10]:
# Define the LLM (default is da-vinci-0.0.3)
da_vinci_llm = OpenAI(openai_api_key=OPENAI_API_KEY)

In [11]:
# Define the RAG pipeline
da_vinci_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()} 
                 | prompt 
                 | da_vinci_llm 
                 | StrOutputParser())

In [12]:
# Test a query
# da_vinci_chain.invoke(queries[1]["text"])

In [13]:
#### API CALL WARNING #####

# Run RAG pipeline for every query
answers_da_vinci = {}
for query_id in tqdm(rel_set.keys(), desc="Asking Queries to ChatGPT with RAG"):
    response = da_vinci_chain.invoke(queries[query_id]["text"])
    answers_da_vinci[query_id] = {"response": response}

Asking Queries to ChatGPT with RAG:   0%|                             | 0/76 [00:05<?, ?it/s]


In [14]:
# Sanity check
queries[1]["text"], answers_da_vinci[1]["response"]

(' What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?',
 '\nThere are a variety of problems and concerns in making up descriptive titles for articles. Authors must consider the type and amount of information that they are trying to communicate and how best to communicate it in fewer words. They must also be aware of keywords that may be used for retrieval and of the various synonyms and related terms that could be used. Automatically retrieving articles from approximate titles can be difficult as titles may not contain all of the key words used in an article or may contain words that are not related to the content. The usual relevance of the content of articles to their titles varies greatly depending on the field of study. Generally, titles contain between one-third and one-half of indexable terms. Howev

In [15]:
# Save all responses
with open("../responses/da-vinci-0.0.3/llm_w_rag_faiss.pkl", "wb") as f:
    pickle.dump(answers_da_vinci, f)

### gpt-3.5-turbo-instruct

In [16]:
# Define the LLM
gpt_llm = OpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-3.5-turbo-instruct")

In [17]:
# Define the RAG pipeline
gpt_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}  
             | prompt 
             | gpt_llm 
             | StrOutputParser())

In [18]:
# Test a query
# gpt_chain.invoke(queries[1]["text"])

In [19]:
#### API CALL WARNING #####

# Run RAG pipeline for every query
answers_gpt = {}
for query_id in tqdm(rel_set.keys(), desc="Asking Queries to ChatGPT with RAG"):
    response = gpt_chain.invoke(queries[query_id]["text"])
    answers_gpt[query_id] = {"response": response}

Asking Queries to ChatGPT with RAG:   0%|                             | 0/76 [00:04<?, ?it/s]


In [20]:
# Sanity check
queries[1]["text"], answers_gpt[1]["response"]

(' What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?',
 '\nAnswer: There are several problems and concerns in making up descriptive titles for articles in engineering literature. One issue is the proportion of information that is contained in the complete document but cannot be deduced from the title alone. This means that even if a title is descriptive and informative, there may still be important information missing that is only found in the full document. Additionally, there are challenges in dealing with synonyms and syntactical variants in titles when searching indexes.\n\nAnother difficulty is the effectiveness of titles in conveying the relevance of an article to potential users. While titles are important for alerting and information services, they may not accurately represent the content of the 

In [21]:
# Save all responses
with open("../responses/gpt-3.5-turbo-instruct/llm_w_rag_faiss.pkl", "wb") as f:
    pickle.dump(answers_gpt, f)

### llama-7b

In [22]:
# Define the LLM
llama_llm = LlamaCpp(
    model_path="../../experiments/backups/llama.cpp/models/llama-2-7b-chat.ggmlv3.q4_K_S.gguf.bin", 
    n_ctx=4096, 
    verbose=False)
llama_llm.client.verbose = False

llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from ../../experiments/backups/llama.cpp/models/llama-2-7b-chat.ggmlv3.q4_K_S.gguf.bin (version unknown)
llama_model_loader: - tensor    0:                token_embd.weight q4_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:               output_norm.weight f32      [  4096,     1,     1,     1 ]
llama_model_loader: - tensor    2:                    output.weight q6_K     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_q.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.attn_k.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:              blk.0.attn_v.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    6:         blk.0.attn_output.weight q4_K     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    7:           blk.0.attn_norm.w

In [23]:
# Define the RAG pipeline
llama_chain = ({"context": retriever | format_docs, "question": RunnablePassthrough()}  
              | prompt 
              | llama_llm 
              | StrOutputParser())

In [24]:
# Test a query
# llama_chain.invoke(queries[1]["text"])

In [25]:
# Run RAG pipeline for every query
answers_llama = {}
for query_id in tqdm(rel_set.keys(), desc="Asking Queries to ChatGPT with RAG"):
    response = llama_chain.invoke(queries[query_id]["text"])
    answers_llama[query_id] = {"response": response}

Asking Queries to ChatGPT with RAG:   0%|                             | 0/76 [02:13<?, ?it/s]


In [26]:
# Sanity check
queries[1]["text"], answers_llama[1]["response"]

(' What problems and concerns are there in making up descriptive titles? What difficulties are involved in automatically retrieving articles from approximate titles? What is the usual relevance of the content of articles to their titles?',
 "\nAnswer:  Based on the given texts, there are several problems and concerns related to creating descriptive titles, including:\n\n1. Information loss: A significant portion of the information contained in an article may not be retrievable from its title alone, leading to a potential mismatch between the title's content and the article's actual content (Brown et al., 1983).\n2. Synonyms and related terms: Using keywords in journal titles as supplementary retrieval tags can help improve the relevance of articles to their titles (Cronin, 1986). However, dealing with synonyms and syntactical variants in searching title indexes remains a challenge.\n3. Title length: Shorter titles may not convey enough information, leading to reduced retrieval efficien

In [27]:
# Save all responses
with open("../responses/llama-7b/llm_w_rag_faiss.pkl", "wb") as f:
    pickle.dump(answers_llama, f)